# Deep Learning with Torch - Tutorial 5

## Learning language models with recurrent networks


First, we need to handle our data. We will use a text file and load it at character level.

We also need to encode the strings into indexed symbols (our vocabulary)

In [1]:

function loadTextFileChars(filename, vocab)
  local file = torch.DiskFile(filename, 'r')
  file:seekEnd()
  local length = file:position() - 1
  file:seek(1)
  local byteVec = torch.ByteTensor(length)
  file:readByte(byteVec:storage())

  local vocab = vocab or {}
  local currentNum = 1
  local data = byteVec:data()
  for i=0, length-1 do
    local encodedNum = vocab[data[i]]
    if not encodedNum then
      vocab[data[i]] = currentNum
      encodedNum = currentNum
      currentNum = currentNum + 1
    end
    data[i] = encodedNum
  end
  local decoder = {}
  for val, num in pairs(vocab) do
    decoder[num] = string.char(val)
  end
  return byteVec, vocab, decoder
end


function reshapeData(wordVec, seqLength, batchSize)
  local offset = offset or 0
  local length = wordVec:nElement()
  local numBatches = torch.floor(length / (batchSize * seqLength))

  local batchWordVec = wordVec.new():resize(numBatches, batchSize, seqLength)
 
  local endIdxs = torch.LongTensor()
  for i=1, batchSize do
    local startPos = torch.round((i - 1) * length / batchSize ) + 1
    local sliceLength = seqLength * numBatches
    batchWordVec:select(2,i):copy(wordVec:narrow(1, startPos, sliceLength))
  end
return batchWordVec
end


function decodeFunc(decoder)
  local space = ''
  local func = function(vec)
    local output = ''
    for i=1, vec:size(1) do
        output = output .. space .. decoder[vec[i]]
    end
    return output
  end
  return func
end


 function encodeFunc(vocab)
    local func = function(str)
        local length = #str
        local encoded = torch.ByteTensor(length):zero()

        for i=1, length do
          local encodedNum = vocab[str[i]]
          if not encodedNum then
              encodedNum = -1
            end
            encoded[i] = encodedNum
          end
          return encoded
    end
  
    return func
end

In [28]:
byteVec, vocab, decoder = loadTextFileChars('hebrewBible.txt')
--byteVec, vocab, decoder = loadTextFileChars('tinyshakespeare.txt')
vocabSize = #decoder
encodeTensor = encodeFunc(vocab)
decodeString = decodeFunc(decoder)

In [29]:
print(vocabSize)
print(byteVec:size())
print(decodeString(byteVec:narrow(1,1,1000)))


39	
 3215738
[torch.LongStorage of size 1]

א,א בראשית, ברא אלוהים, את השמיים, ואת הארץ.  א,ב והארץ, הייתה תוהו ובוהו, וחושך, על-פני תהום; ורוח אלוהים, מרחפת על-פני המים.  א,ג ויאמר אלוהים, יהי אור; ויהי-אור.  א,ד וירא אלוהים את-האור, כי-טוב; ויבדל אלוהים, בין האור ובין החושך.  א,ה ויקרא אלוהים לאור יום, ולחושך קרא לילה; ויהי-ערב ויהי-בוקר, יום אחד.  {פ}

א,ו ויאמר אלוהים, יהי רקיע בתוך המים, ויהי מבדיל, בין מים למים.  א,ז ויעש אלוהים, את-הרקיע, ויבדל בין המים אשר מתחת לרקיע, ובין המים אשר מעל לרקיע; ויהי-כן.  א,ח ויקרא אלוהים לרקיע, שמיים; ויהי-ערב ויהי-בוקר, יום שני.  {פ}

א,ט ויאמר אלוהים, ייקוו המים מתחת השמיים אל-מ	


In [4]:
batchSize = 50
seqLength = 20
trainData = reshapeData(byteVec, seqLength + 1, batchSize)
print(trainData:size())

 3062
   50
   21
[torch.LongStorage of size 3]



We will use the ```recurrent``` package for torch.
https://github.com/eladhoffer/recurrent.torch


A popular alternative is the ```rnn``` package
https://github.com/Element-Research/rnn

In [26]:
require 'recurrent'
local inputSize = 5
local outputSize = 7

local rnn = nn.RNN(inputSize, outputSize)
local lstm = nn.LSTM(inputSize, outputSize)
local gru = nn.GRU(inputSize, outputSize)


print('----RNN - Single----')
-- To feed single time step, we use 'single' mode
local batch = 16
local x = torch.rand(batch, inputSize)
rnn:single()

-- And now we can feed the input to get output. We can also peek at state
local y = rnn:forward(x)
print('Output size: '); print(y:size())
print('State size: '); print(rnn:getState():size())


print('----LSTM - Sequence----')
--We mostly feed sequences through rnns
local timeLength = 20
x = torch.rand(batch, timeLength, inputSize)

-- We will enable this by putting the recurrent model to 'sequence' mode
lstm:sequence()

-- And now we can feed the sequence to get output. We can also peek at state
local y = lstm:forward(x)
print('Output size: '); print(y:size())
print('State size: '); print(lstm:getState():size())




----RNN - Single----	
Output size: 	
 16
  7
[torch.LongStorage of size 2]

State size: 	
 16
  7
[torch.LongStorage of size 2]

----LSTM - Sequence----	


Output size: 	
 16
 20
  7
[torch.LongStorage of size 3]

State size: 	
 16
 14
[torch.LongStorage of size 2]



Non recurrent layers and criterions are wrapped in ```TemporalModule``` and ```TemporalCriterion``` containers

In [5]:
require 'recurrent'
require 'cunn'

rnnSize = 256
embedder = nn.LookupTable(vocabSize, rnnSize)
classifier = nn.Linear(rnnSize, vocabSize)
rnn = nn.LSTM(rnnSize, rnnSize)

model = nn.Sequential()
model:add(embedder)
model:add(rnn)
model:add(nn.TemporalModule(classifier))
model:add(nn.TemporalModule(nn.LogSoftMax()))


model:cuda()
model:sequence()
embedder:share(classifier, 'weight', 'gradWeight')
w, dE_dw = model:getParameters()
print('#Parameters = ', w:nElement())

criterion = nn.TemporalCriterion(nn.ClassNLLCriterion()):cuda()

w:uniform(-0.08, 0.08)

#Parameters = 	535335	


### Training the network

In [6]:
require 'optim'

Vmax = 5
optimState = {learningRate = 1e-3}


function forwardNet(data, train)
    local lossAcc = 0
    local numBatches = data:size(1)
    model:sequence()
    model:forget()
    model:zeroState()
    if train then
        --set network into training mode
        model:training()
    else
        model:evaluate()
    end
    local x = torch.CudaTensor(batchSize, seqLength)
    local yt = torch.CudaTensor(batchSize, seqLength)
    for i = 1, numBatches do
        -- our input and target are the same sequence shifted
        x:copy(data[i]:narrow(2, 1, seqLength))
        yt:copy(data[i]:narrow(2, 2, seqLength))
        
        
        local y = model:forward(x)
        local err = criterion:forward(y, yt)

        lossAcc = lossAcc + err / seqLength
        
        
        if train then
            function feval()
                model:zeroGradParameters() --zero grads
                local dE_dy = criterion:backward(y,yt)
                model:backward(x, dE_dy) -- backpropagation
                local norm = dE_dw:norm()
                if norm > Vmax then -- gradient renormalization to avoid explosion
                    local shrink = Vmax / norm
                    dE_dw:mul(shrink)
                end
                return err, dE_dw
            end
        
            optim.adam(feval, w, optimState)
        end
    end
    

    local avgLoss = lossAcc / numBatches
    
    
    return avgLoss
end


We'll also introduce a sampling function - to try and generate sequences from our network

In [27]:

function sample(num, space, temperature)
    local num = num or 50
    local temperature = temperature or 1
    local function smp(preds)
        if temperature == 0 then
            local _, num = preds:max(2)
            return num
        else
            preds:div(temperature) -- scale by temperature
            local probs = preds:squeeze()
            probs:div(probs:sum()) -- renormalize so probs sum to one
            local num = torch.multinomial(probs:float(), 1):typeAs(preds)
            return num
        end
    end

    local sampleModel = nn.Sequential():add(embedder):add(rnn):add(classifier):add(nn.SoftMax():cuda())

    sampleModel:evaluate()
    sampleModel:single()

    
    local pred, predText, embedded
    
    wordNum = torch.Tensor(1):random(vocabSize):cuda()

    local predText = {}
    
    for i=1, num do
        pred = sampleModel:forward(wordNum)
        wordNum = smp(pred)
        predText[i] = wordNum:squeeze()
    end
    return torch.Tensor(predText)
end

In [7]:
epochs = 5
trainLoss = torch.Tensor(epochs)

print('Sampled text:')
print(decodeString(sample(500)))
for e = 1, epochs do
  print('Epoch ' .. e .. ':')
  trainLoss[e] = forwardNet(trainData, true)
    
  print('Training Loss: ' .. trainLoss[e])
    
  print('Sampled text:')
  print(decodeString(sample(500)))
end

Sampled text:	


Training Loss: 1.2775420697977	
Sampled text:	


ר-הארץ;    גיקר; והשם בין-גוליים תצחבה-אל-תשלחנו בארץ-אשר:  המזבקת חמר מבצר, התאקרה אשר באלמים ואיש, מיה; לשמח-גמתיהם, לך הסהב וכדור, ומלך-הזאקרוח.
עב,א  גשלו, מפני-יהוד תעללים כי-עוזה, מפני יהובני המלכישלגא, והיה רבחנה לירחנה:  כי-ראשי, נתבורו ידון אל-רגבל יעקול, והעיר ליהוה.  ב,יא חזק	
Epoch 2:	


Training Loss: 1.1135815120382	
Sampled text:	


Training Loss: 1.0722586075166	
Sampled text:	


 דיבר-יהוה--נדים, מקומו--טמא, על-בני שלכים, ישראל ישראל, מאהב בבל; ויעזבם, מי לאלה לאמור:  את כל-הוא וחושם מצוהרי, סביבות את-כל-גיד המלך משילות, אשר שבע צסון--מלך-פדר; והנפש את-כל-דרכו, מזמור--בחוץ הנשמר, במלך:  בתית לו שקר הנביא הראשון, להם בנו-החצר הדבר, אשר יקרא, עוונם הגונינים. ב ד,טו 	
Epoch 4:	


Training Loss: 1.0446834487124	
Sampled text:	


Training Loss: 1.0277193483771	
Sampled text:	


קני יהוה.  נא,כב אז אל-הקשת כי-תוכיח על-הפקודים--את-שלושה נמוצדים הניחמם איש-אחאב, מלץ; וישאו בת-אמור, אבשמו ארץ גדול.  מז,יז לנשא, עליו, וייפול ודויד.  ס}

כב,ח טופג על-חואני--ברכו שפרה כתף, חילו מלך אביו; ותהיו מכים זהב, {ס}  בעימתי, וצבא שאול, מיניו לבבל, כחרב מלחם עלי כל-הכריס; ומבנות נ	


In [8]:
 print('Sampled text:')
  print(decodeString(sample(500)))

Sampled text:	


 נולוה.  {ס}

כג,א בשמן בדי-שדה, עליי אל-מימינה מותנו.
קב,ג  כי-עמד ראפו שמונה דמיקים תבוכו;    ולבנך טובה, בושני יעשו    נלשכם, תבקר היית;    ולך מחטאו במושו; - אם-אמרים שם חמדו נשינו, חושש תרביבני;   אנוכי יתאול, יחדיתי שתיה,    פתח יהוה;   וארון ישר, כי-הוא, הקהלתו.
קמא,ב  קין מצדקה תיקח יהוה-	
